CLASIFICADOR


In [53]:
import pandas as pd
from tkinter import Tk, filedialog

# Función para seleccionar archivos
def seleccionar_archivo():
    root = Tk()
    root.withdraw()  # Oculta la ventana de Tkinter
    archivo = filedialog.askopenfilename(title="Selecciona un archivo CSV", filetypes=[("Archivos CSV", "*.csv")])
    return archivo

# Cargar bases de datos
print("Selecciona la base de datos de libros:")
archivo_libros = seleccionar_archivo()
print("Selecciona el historial de usuario:")
archivo_historial = seleccionar_archivo()

df_libros = pd.read_csv(archivo_libros, quotechar='"', on_bad_lines='skip')
df_historial = pd.read_csv(archivo_historial)

# Limpieza de columnas
df_libros.columns = df_libros.columns.str.strip()
df_libros.columns = df_libros.columns.str.replace('  ', ' ')

# Filtrar libros que están en el historial
libros_historial = df_libros[df_libros["bookID"].isin(df_historial["bookID"])].copy()

# Excluir libros que están en el historial de las recomendaciones
recomendaciones = df_libros[~df_libros["bookID"].isin(libros_historial["bookID"])]

# Filtrar por idioma del historial
idiomas_historial = libros_historial['language_code'].unique()
print(f"Idiomas en el historial del usuario: {idiomas_historial}")
recomendaciones = recomendaciones[recomendaciones['language_code'].isin(idiomas_historial)]

# Priorizar idiomas más leídos
def priorizar_idiomas(df, idiomas_frecuencia):
    df = df.copy()  # Crear una copia explícita del DataFrame
    df.loc[:, 'idioma_score'] = df['language_code'].map(idiomas_frecuencia)
    return df.sort_values(by=['idioma_score', 'average_rating'], ascending=[False, False])

# Crear frecuencia de idiomas
idiomas_frecuencia = libros_historial['language_code'].value_counts(normalize=True).to_dict()
recomendaciones = priorizar_idiomas(recomendaciones, idiomas_frecuencia)

# Relajar el filtro por promedio de páginas
promedio_paginas = libros_historial['num_pages'].mean()
desviacion_paginas = libros_historial['num_pages'].std()
recomendaciones = recomendaciones[
    (recomendaciones['num_pages'] >= promedio_paginas - (2 * desviacion_paginas)) & 
    (recomendaciones['num_pages'] <= promedio_paginas + (2 * desviacion_paginas))
]

# Relajar el filtro por reseñas
recomendaciones = recomendaciones[
    (recomendaciones['average_rating'] >= 3.0) & 
    (recomendaciones['ratings_count'] > 10)
]

# Asegurar representación de todos los idiomas del historial
def balancear_idiomas(df, idiomas_historial, min_por_idioma=5):
    df_balanceado = pd.DataFrame()
    for idioma in idiomas_historial:
        subset = df[df['language_code'] == idioma]
        if len(subset) < min_por_idioma:
            df_balanceado = pd.concat([df_balanceado, subset])
        else:
            df_balanceado = pd.concat([df_balanceado, subset.head(min_por_idioma)])
    return df_balanceado

recomendaciones = balancear_idiomas(recomendaciones, idiomas_historial)

# Ajustar el umbral para obtener entre 50 y 100 recomendaciones
while len(recomendaciones) > 100:
    recomendaciones = recomendaciones.sample(frac=0.9, random_state=42)
if len(recomendaciones) < 50:
    print("Aumentando el umbral para obtener más recomendaciones...")
    adicionales = df_libros[
        (df_libros['average_rating'] >= 2.5) & 
        (df_libros['ratings_count'] > 5) & 
        (~df_libros['bookID'].isin(recomendaciones['bookID'])) & 
        (df_libros['language_code'].isin(idiomas_historial))
    ]
    adicionales = priorizar_idiomas(adicionales, idiomas_frecuencia)
    recomendaciones = pd.concat([recomendaciones, adicionales]).drop_duplicates()
    recomendaciones = recomendaciones.head(100)

# Verificar que ningún libro en las recomendaciones esté en el historial
recomendaciones = recomendaciones[~recomendaciones["bookID"].isin(libros_historial["bookID"])]

# Guardar la base de datos de recomendaciones finales
nombre_archivo = "recomendaciones_finales.csv"
recomendaciones.to_csv(nombre_archivo, index=False)
print(f"\nBase de datos de recomendaciones finales guardada como '{nombre_archivo}'.")

# Mostrar un resumen de las recomendaciones
print("\nResumen de las recomendaciones finales:")
print(recomendaciones[["title", "authors", "average_rating", "num_pages", "ratings_count", "language_code"]].head(50))


Selecciona la base de datos de libros:
Selecciona el historial de usuario:
Idiomas en el historial del usuario: ['eng' 'en-US' 'spa']
Aumentando el umbral para obtener más recomendaciones...

Base de datos de recomendaciones finales guardada como 'recomendaciones_finales.csv'.

Resumen de las recomendaciones finales:
                                                   title  \
4373                              Existential Meditation   
6587                      The Complete Calvin and Hobbes   
6589       It's a Magical World (Calvin and Hobbes  #11)   
1848                                         Early Color   
6590   Homicidal Psycho Jungle Cat (Calvin and Hobbes...   
4811            The Feynman Lectures on Physics Vols 7-8   
4810            The Feynman Lectures on Physics Vols 3-4   
7042   The Sibley Field Guide to Birds of Western Nor...   
6196   Discovery of the Presence of God: Devotional N...   
1611             The Feynman Lectures on Physics  3 Vols   
11295                

ALGORITMO GENETICO 

In [160]:
import pandas as pd
import csv
from collections import defaultdict
import random

def calcular_puntuaciones(historial_path):
    author_count = defaultdict(int)
    language_count = defaultdict(int)

    with open(historial_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            authors = row['authors'].split('/')
            for author in authors:
                author_count[author.strip()] += 1
            language_count[row['language_code'].strip()] += 1

    max_author_count = max(author_count.values())
    max_language_count = max(language_count.values())

    author_scores = {author: (count / max_author_count) * 10 for author, count in author_count.items()}
    language_scores = {language: (count / max_language_count) * 10 for language, count in language_count.items()}

    return author_scores, language_scores

def inicializacion_random_poblacion(recomendaciones, n, subset_size):
    poblacion = []
    for _ in range(n):
        subset = recomendaciones.sample(n=subset_size, replace=False).to_dict(orient='records')
        poblacion.append(subset)
    return poblacion

def calcular_aptitud(subset, author_scores, language_scores):
    avg_rating = sum([libro['average_rating'] for libro in subset]) / len(subset)
    author_score = sum([author_scores.get(libro['authors'].split('/')[0].strip(), 0) for libro in subset]) / len(subset)
    language_score = sum([language_scores.get(libro['language_code'], 0) for libro in subset]) / len(subset)
    return 0.9 * avg_rating + 5 * author_score + 1.5 * language_score

def seleccion_ruleta(poblacion, aptitudes):
    total_aptitud = sum(aptitudes)
    seleccionados = []
    while len(seleccionados) < len(poblacion):
        pick = random.uniform(0, total_aptitud)
        acumulado = 0
        for i, aptitud in enumerate(aptitudes):
            acumulado += aptitud
            if acumulado >= pick:
                seleccionados.append(poblacion[i])
                break
    if len(seleccionados) % 2 != 0:
        seleccionados.pop()
    return seleccionados

def cruzar_padres(padre1, padre2):
    punto_cruce = random.randint(1, len(padre1) - 1)
    hijo1 = padre1[:punto_cruce] + padre2[punto_cruce:]
    hijo2 = padre2[:punto_cruce] + padre1[punto_cruce:]
    return hijo1, hijo2


def mutar(hijo, recomendaciones, libros_historial):
    if random.uniform(0, 1) <= 0.1:  
        indice = random.randint(0, len(hijo) - 1)
        titulos_usados = {libro['title'] for libro in hijo}
        titulos_usados.update(libros_historial['title'].tolist())
        
        nuevo_libro = recomendaciones.sample(n=1).to_dict(orient='records')[0]
        while nuevo_libro['title'] in titulos_usados:
            nuevo_libro = recomendaciones.sample(n=1).to_dict(orient='records')[0]
        
        hijo[indice] = nuevo_libro  
    return hijo


def completar_subconjunto(subset, recomendaciones, libros_historial, subset_size):
   
    titulos_usados = {libro['title'] for libro in subset}
    titulos_usados.update(libros_historial['title'].tolist())
    
    while len(subset) < subset_size:
        nuevo_libro = recomendaciones.sample(n=1).to_dict(orient='records')[0]
        if nuevo_libro['title'] not in titulos_usados:
            subset.append(nuevo_libro)
            titulos_usados.add(nuevo_libro['title']) 
    return subset

def algoritmo_genetico(recomendaciones, author_scores, language_scores, libros_historial, generaciones=50, poblacion_size=10, subset_size=5):
    poblacion = inicializacion_random_poblacion(recomendaciones, poblacion_size, subset_size)
    
    for gen in range(generaciones):
        print(f"\nGeneración {gen + 1}:")
        
        for i, subset in enumerate(poblacion):
            print(f"Individuo {i + 1}: {[libro['title'] for libro in subset]}")
        
        aptitudes = [calcular_aptitud(subset, author_scores, language_scores) for subset in poblacion]
        print(f"Aptitudes: {aptitudes}")
        
        padres = seleccion_ruleta(poblacion, aptitudes)

        nueva_generacion = []
        for i in range(0, len(padres) - 1, 2):
            ran = random.uniform(0, 1)
            if ran <= 0.85:
                hijo1, hijo2 = cruzar_padres(padres[i], padres[i+1])
                hijo1 = mutar(hijo1, recomendaciones, libros_historial)
                hijo2 = mutar(hijo2, recomendaciones, libros_historial)
            else:
                hijo1 = padres[i]
                hijo2 = padres[i+1]

            hijo1 = completar_subconjunto(hijo1, recomendaciones, libros_historial, subset_size)
            hijo2 = completar_subconjunto(hijo2, recomendaciones, libros_historial, subset_size)

            nueva_generacion.extend([hijo1, hijo2])
        
        poblacion = nueva_generacion
    
    aptitudes_finales = [calcular_aptitud(subset, author_scores, language_scores) for subset in poblacion]
    mejor_solucion = poblacion[aptitudes_finales.index(max(aptitudes_finales))]
    return mejor_solucion

historial_path = 'historial_usuario.csv'
recomendaciones_path = 'recomendaciones_finales.csv'

recomendaciones = pd.read_csv(recomendaciones_path)
libros_historial = pd.read_csv(historial_path)

author_scores, language_scores = calcular_puntuaciones(historial_path)

try:
    mejor_subconjunto = algoritmo_genetico(recomendaciones, author_scores, language_scores, libros_historial, generaciones=50, poblacion_size=10, subset_size=5)
    print("\nMejor subconjunto de libros recomendado:")
    for libro in mejor_subconjunto:
        print(libro)
except ValueError as e:
    print(f"Error en el algoritmo genético: {e}")





Generación 1:
Individuo 1: ['The Shawshank Redemption: The Shooting Script', 'The Sibley Field Guide to Birds of Western North America', '100 Years of Lynchings', 'Occaecati quasi impedit nam sapiente modi', 'Cocolat: Extraordinary Chocolate Desserts']
Individuo 2: ['The Complete Calvin and Hobbes', 'A Guide to the Words of My Perfect Teacher', 'The Lord of the Rings: The Art of the Fellowship of the Ring', 'The Absolute Sandman  Volume One', 'Occaecati quasi impedit nam sapiente modi']
Individuo 3: ['Tempore quo magni temporibus', 'Fullmetal Alchemist  Vol. 14 (Fullmetal Alchemist  #14)', 'The Sawtooth Wolves', 'Getting a Grip on the Basics: Building a Firm Foundation for the Victorious Christian Life', 'The Feynman Lectures on Physics Vol 3']
Individuo 4: ['Provident vel praesentium voluptas nihil', 'Harry Potter and the Half-Blood Prince (Harry Potter  #6)', 'Fullmetal Alchemist  Vol. 6 (Fullmetal Alchemist  #6)', 'Jane Austen: The Complete Novels', 'Nausicaä of the Valley of the W